Insert explanatory text here. Insert timing here.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of a specific $(2,1)$-hypersurface $X_1$ in $\mathbb{P}^1 \times \mathbb{P}^2$.

In [2]:
F = GF(2)
P.<x0,x1,y0,y1,y2> = F[]
gen1 = x0^2*y0 + x0*x1*y1 + x1^2*y2

In [3]:
S1 = [vector(F,t) for t in itertools.product(F, repeat=2) if not all(i==0 for i in t)]
S2 = [vector(F,t) for t in itertools.product(F, repeat=3) if not all(i==0 for i in t)]
for v in S1 + S2:
    v.set_immutable()
S0 = list(itertools.product(S1, S2))
S = [x for x in S0 if gen1(*x[0], *x[1]) == 0]
len(S)

9

Construct the sets of $\mathbb{F}_4$-points and $\mathbb{F}_8$-points of $X_1$.

In [4]:
F4 = GF(4)
S14 = [vector(F4,t) for t in itertools.product(F4, repeat=2) if not all(i==0 for i in t) and [i for i in t if i != 0][0] == 1]
S24 = [vector(F4,t) for t in itertools.product(F4, repeat=3) if not all(i==0 for i in t) and [i for i in t if i != 0][0] == 1]
for v in S14 + S24:
    v.set_immutable()
S04 = list(itertools.product(S14, S24))
S4 = [x for x in S04 if gen1(*x[0], *x[1]) == 0]
len(S4)

25

In [5]:
F8 = GF(8)
S18 = [vector(F8,t) for t in itertools.product(F8, repeat=2) if not all(i==0 for i in t) and [i for i in t if i != 0][0] == 1]
S28 = [vector(F8,t) for t in itertools.product(F8, repeat=3) if not all(i==0 for i in t) and [i for i in t if i != 0][0] == 1]
for v in S18 + S28:
    v.set_immutable()
S08 = list(itertools.product(S18, S28))
S8 = [x for x in S08 if gen1(*x[0], *x[1]) == 0]
len(S8)

81

Construct a subgroup $G$ of $\mathrm{GL}(2, \mathbb{F}_2) \times \mathrm{GL}(3, \mathbb{F}_2)$ fixing $X_1$.

In [6]:
l0 = [Matrix(F,[[1,1,0,0,0],[0,1,0,0,0],[0,0,1,0,0],[0,0,0,1,0],[0,0,1,1,1]]),
      Matrix(F,[[0,1,0,0,0],[1,0,0,0,0],[0,0,0,0,1],[0,0,0,1,0],[0,0,1,0,0]])]
G0 = GL(5,F).subgroup(l0)
G0.order()

6

Use an orbit lookup tree to find $G$-orbit representatives for 6-tuples of $\mathbb{F}_2$-points in $X_1$.

In [7]:
def apply_group_elem(g, x):
    g1 = g.submatrix(nrows=2,ncols=2)
    g2 = g.submatrix(row=2,col=2)
    v1 = g1*x[0]
    v2 = g2*x[1]
    v1.set_immutable()
    v2.set_immutable()
    return (v1, v2)

In [8]:
assert all(apply_group_elem(g, x) in S for g in l0[:1] for x in S)

In [9]:
def stabilizer(x):
    G1 = vec_stab(Matrix(F, x[0]), transpose=True)
    G2 = vec_stab(Matrix(F, x[1]), transpose=True)
    l0 = [block_matrix(2,2,[g.matrix(),0,0,identity_matrix(3)], subdivide=False) for g in G1.gens()] + \
        [block_matrix(2,2,[identity_matrix(2),0,0,g.matrix()], subdivide=False) for g in G2.gens()]
    return GL(5, F).subgroup(l0)

In [10]:
def optimized_rep(g):
    return g.matrix()

In [11]:
methods = {'apply_group_elem': apply_group_elem,
           'stabilizer': stabilizer,
           'optimized_rep': optimized_rep}
tree = build_orbit_tree(G0, S, 6, methods, verbose=False)

In [12]:
[len(green_nodes(tree, s)) for s in range(4, 7)]

[24, 24, 17]

For $k=4,5,6$, for each $k$-tuple of $\mathbb{F}_2$-rational points of $X_1$, find $(2,3)$-surfaces $X_2$ such that $X_1 \cap X_2$ contains precisely the chosen set of $\mathbb{F}_2$-points. We also enforce the desired point counts over $\mathbb{F}_4$.

In [13]:
monos02 = [prod(x) for x in itertools.combinations_with_replacement([y0,y1,y2], 2)]
#monos22 = [prod(x) for x in itertools.product([x0^2,x0*x1,x1^2],[y0^2,y0*y1,y1^2,y0*y2,y1*y2,y2^2])]
monos23 = [prod(x) for x in itertools.product([prod(y) for y in itertools.combinations_with_replacement([x0,x1],2)],
                                              [prod(y) for y in itertools.combinations_with_replacement([y0,y1,y2],3)])]
#monos43 = [prod(x) for x in itertools.product([prod(y) for y in itertools.combinations_with_replacement([x0,x1],4)],
#                                              [prod(y) for y in itertools.combinations_with_replacement([y0,y1,y2],3)])]
len(monos23)

30

In [14]:
def redundancy(x, F=F, monos02=monos02, monos23=monos23):
    return [vector(F, ((gen1*mu).coefficient(mu2) for mu2 in monos23)) for mu in monos02]

len(redundancy(x))

6

In [15]:
coords23 = {x: vector(F, (mu(*x[0], *x[1]) for mu in monos23)) for x in S}

In [ ]:
perp = Matrix([coords23[x] for x in S])
curves = defaultdict(list)
V = VectorSpace(F, len(monos23))
tmp2 = [t[:2] for t in targets6]
tmp3 = [t[:3] for t in targets6]
for s in range(4,7):
    for vecs in green_nodes(tree, s):
#        print(s, vecs)
        target = vector(F, (0 if x in vecs else 1 for x in S))
        for v2 in solve_right_iterator(perp, target, redundancy, None):
            gen2 = sum(v2[i]*monos23[i] for i in range(len(monos23)))
            s2 = sum(1 for x in S4 if gen2(*x[0], *x[1]) == 0)
            if (s,s2) in tmp2:
                s3 = sum(1 for x in S8 if gen2(*x[0], *x[1]) == 0)
                if (s,s2,s3) in tmp3:
                    curves[(s,s2,s3)].append((gen1, gen2))

In [ ]:
[(s, len(curves[s])) for s in curves]

Compute point counts over $\mathbb{F}_{q^i}$ for $i=4,5$, retaining cases that fit our targets.

In [ ]:
def count_by_ideal(gens, n):
    J = P.ideal(list(gens) + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 2^(2*n) - 2^(3*n) + 1) // (2^n-1)^2

In [ ]:
for n in range(4, 6):
    tmp = [t[:n] for t in targets6]
    tmp2 = list(curves.keys())    
    for s in tmp2:
        if len(s) == n-1:
            for gens in curves[s]:
                i = count_by_ideal(gens, n)
                s1 = s + (i,)
                if s1 in tmp:
                    curves[s1].append(gens)
    print([(s, len(curves[s])) for s in curves if len(s) == n])

Use Magma to compute the full zeta functions of the resulting curves (discarding cases that do not lead to smooth curves of the right genus), and compare these to our targets.

In [ ]:
I1 = P.ideal([x0,x1])
I2 = P.ideal([y0,y1,y2])
CR = magma.CoxRing(P, [I1,I2], [[1,1,0,0,0],[0,0,1,1,1]], [])
proj = CR.ToricVariety()
proj

In [ ]:
l = []
for s in curves:
    if len(s) == 4:
        for gens in curves[s]:
            X = proj.Scheme(gens)
            if X.Dimension() > 1:
                continue
            C = X.Curve()
            g = C.Genus()
            assert g <= 6
            if g != 6:
                continue
            ct = tuple(Integer(C.NumberOfPlacesOfDegreeOneECF(i)) for i in range(1, 7))
            if ct in targets7:
                l.append(C)
len(l)

In [ ]:
l = isomorphism_class_reps(l)
len(l)

Search for extensions of relative class number 1.

In [ ]:
l2 = []
for C in l:
    F0 = C.FunctionField()
    F1 = F0.AlgorithmicFunctionField()
    l2 += match_weil_poly(F1, 2)
len(l2)

In [ ]:
for (F1, F2) in l2:
    t = tuple(F1.NumberOfPlacesOfDegreeOneECF(i) for i in range(1,7))
    print(t, F1.RationalExtensionRepresentation())

In [ ]:
print("This case complete!")